In [2]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.models.transforms.outcome import Standardize
from botorch.models.transforms.input import Normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import ExpectedImprovement
from botorch.acquisition.analytic import LogExpectedImprovement
from botorch.acquisition import qLogExpectedImprovement
from botorch.optim import optimize_acqf
from gpytorch.likelihoods import GaussianLikelihood
import gpytorch
import win32com.client
from botorch.models.transforms import Normalize
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.metrics import mean_absolute_error
from scipy.stats import qmc
import os
import subprocess
from pathlib import Path
import time
import pandas as pd
import plotly.graph_objects as go
from utils import parse_real_disp

torch.set_default_dtype(torch.float64)

ImportError: cannot import name 'parse_real_disp' from 'utils' (h:\moldflow\moldflow_automation\utils.py)

# Random Parameter Generation

In [3]:
# Bounds
n_bounds = (0.0, 1.0)
tau_star_bounds = (5e-5, 5000.0)
B_bounds_log = (np.log10(1.0), np.log10(1.916e+19))  # log10 scaling
tau_w_bounds = (0.001, 1.0)
m_bounds = (0.1, 1.9)

# LHs Sampling
sampler = qmc.LatinHypercube(d=5)
sample = sampler.random(5)  # 5 samples

# Scaling
n_values = sample[:, 0] * (n_bounds[1] - n_bounds[0]) + n_bounds[0]
tau_star_values = sample[:, 1] * (tau_star_bounds[1] - tau_star_bounds[0]) + tau_star_bounds[0]
B_values = 10 ** (sample[:, 2] * (B_bounds_log[1] - B_bounds_log[0]) + B_bounds_log[0])
tau_w_values = sample[:, 3] * (tau_w_bounds[1] - tau_w_bounds[0]) + tau_w_bounds[0]
m_values = sample[:, 4] * (m_bounds[1] - m_bounds[0]) + m_bounds[0]

# save2CSV
df = pd.DataFrame({
    'n': n_values,
    'tau_star': tau_star_values,
    'B': B_values,
    'tau_w': tau_w_values,
    'm': m_values
})
df.to_csv("combined_LHS_parameters.csv", index=False)

# XML Parsing - Master Curve

In [4]:
# Directories
real_disp_path = "H:\\Real_data\\press_displacement_real.xml"
real_force_path = "H:\\Real_data\\press_force_real.xml"

# Displacement XML

real_disp_time, real_disp = parse_real_disp(real_disp_path)

# real_tree = ET.parse(real_disp_path)
# real_root = real_tree.getroot()
# real_data = []
# guess_data_all = {}

# for time_block, dept_block in zip(real_root.findall('.//Block//IndpVar'), real_root.findall('.//Block')):

#     time_value = time_block.attrib.get('Value')
#     dept_value = dept_block.find('DeptValues').text.strip()
#     real_data.append((time_value, dept_value))

# real_data_array = np.array([(float(x), float(y)) for x, y in real_data])
# real_time = real_data_array[:, 0]  
# real_disp = real_data_array[:, 1] * 0.0001019716


# Force XML

real_force_time, real_force = parse_real_force(real_force_path)

NameError: name 'parse_real_disp' is not defined

# Moldflow Simulation

In [ ]:
Synergy = win32com.client.Dispatch("synergy.Synergy")
Synergy.SetUnits("Metric")
moldflow = "C:\\Program Files\\Autodesk\\Moldflow Insight 2026\\bin"                        #Home D:\\Moldflow Insight 2023\\bin       #WS "C:\\Program Files\\Autodesk\\Moldflow Insight 2023\\bin"  #"G:\\Oskay Sözen\\wall_slip\\wall slip.mpi"


Synergy.OpenProject("G:\\Oskay Sözen\\viscosity\\viscosity.mpi")
Project = Synergy.Project
Project.OpenItemByName("50SurfaceCoverage_study", "Study")



# Simulation loop for wall slip parameters
for i in range(5):

    n = n_values[i]
    tau_star = tau_star_values[i]
    B = B_values[i]
    tau_w = tau_w_values[i]
    m = m_values[i]


    print(f"Simulation {i+1} with \nn : {n}\ntau_star : {tau_star}\nB : {B}\ntau_w : {tau_w}\nm : {m}")

    study_name = f"study_{i+1}"
    Project = Synergy.Project

    # Viscosity Parameters

    PropEd = Synergy.PropertyEditor
    Prop = PropEd.FindProperty(10090,1)
    Prop.FieldDescription(53030, f"{n}, {tau_star}, {B}, 0.004, 2.412, 6.272, 0.305")
    PropEd.CommitChanges("Process Conditions")

    # Wall Slip Parameters

    PropEd = Synergy.PropertyEditor
    Prop = PropEd.FindProperty(10090, 1)
    DVec = Synergy.CreateDoubleArray
    
    DVec.AddDouble(tau_w)
    DVec.AddDouble(m)
    DVec.AddDouble(1e-08)
    DVec.AddDouble(0)
    DVec.AddDouble(0)
    
    Prop.FieldValues(51182, DVec)
        
    PropEd.CommitChanges("Process Conditions")

    StudyDoc = Synergy.StudyDoc
    StudyDoc.SaveAs(study_name)

    p = subprocess.Popen(
    [os.path.join(moldflow, "runstudy.exe"), f"G:\\Oskay Sözen\\viscosity\\study_{i+1}.sdy"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    )
    (output, err) = p.communicate()

    time.sleep(3)

    # Restart of the Synergy to prevent XML saving crash
    
    subprocess.run(["taskkill", "/f", "/im", "airsyn_synergy.exe"]) 

    time.sleep(3)

    Synergy.OpenProject("G:\\Oskay Sözen\\viscosity\\viscosity.mpi")         # PROJECT NAME!!!!!!!!!!!!!!!!
    Project = Synergy.Project
    Project.OpenItemByName(f"study_{i+1}", "Study")

    # Plot Export

    PlotMgr = Synergy.PlotManager
    time.sleep(3)
    Plot = PlotMgr.FindPlotByName2("Press force:XY Plot", "Press force")
    Pid = Plot.GetDataID
    PlotMgr = None
    PlotMgr = Synergy.PlotManager
    time.sleep(3)
    PlotMgr.SaveResultDataInXML2( Pid, f"H:\\XMLFiles\\study{i + 1}_force.xml", "Metric")

    PlotMgr = Synergy.PlotManager
    time.sleep(3)
    Plot = PlotMgr.FindPlotByName2("Press displacement:XY Plot", "Press displacement")
    Pid = Plot.GetDataID
    PlotMgr = None
    PlotMgr = Synergy.PlotManager
    time.sleep(3)
    PlotMgr.SaveResultDataInXML2( Pid, f"H:\\XMLFiles\\study_{i + 1}_disp.xml", "Metric")


# Write the output to a log file
    with open(f"H:\\Log_Files\\study_{i+1}.log", "w") as file:
        file.write(output.decode("windows-1252").strip())



# XML Parsing - Guess Curves

In [ ]:
guess_disp_data_all = {}
guess_force_data_all = {}
area_data = []

# Displacement XML

for i in range(5):
    guess_tree = ET.parse(f"H:\\XMLFiles\\study_{i + 1}_disp.xml")
    guess_root = guess_tree.getroot()
    guess_data = []

    for time_block, dept_block in zip(guess_root.findall('.//Block//IndpVar'), guess_root.findall('.//Block')):
            time_value = time_block.attrib.get('Value')
            dept_value = dept_block.find('DeptValues').text.strip()
            guess_data.append((time_value, dept_value))

    guess_disp_data_all[f"guess_data_{i+1}"] = guess_data

    guess_data_array = np.array([(float(x), float(y)) for x, y in guess_data])
    guess_disp_time = guess_data_array[:, 0]  
    guess_disp = guess_data_array[:, 1]

    globals()[f"guess_disp_time_{i+1}"] = guess_disp_time
    globals()[f"guess_disp_{i+1}"] = guess_disp

    mse_disp = CABC_Displacement(real_disp_time, real_disp, guess_disp_time, guess_disp)
    area_data.append(mse_disp)

print(guess_disp_data_all)

# Force XML

for i in range(5):
    guess_tree = ET.parse(f"H:\\XMLFiles\\study_{i + 1}_force.xml")
    guess_root = guess_tree.getroot()
    guess_data = []

    for time_block, dept_block in zip(guess_root.findall('.//Block//IndpVar'), guess_root.findall('.//Block')):
            time_value = time_block.attrib.get('Value')
            dept_value = dept_block.find('DeptValues').text.strip()
            guess_data.append((time_value, dept_value))

    guess_force_data_all[f"guess_data_{i+1}"] = guess_data

    guess_data_array = np.array([(float(x), float(y)) for x, y in guess_data])
    guess_force_time = guess_data_array[:, 0]  
    guess_force = guess_data_array[:, 1]

    globals()[f"guess_force_time_{i+1}"] = guess_force_time
    globals()[f"guess_force_{i+1}"] = guess_force

    mse_disp = CABC_Force(real_disp_time, real_force, guess_force_time, guess_disp)
    area_data.append(mse_disp)

print(guess_force_data_all)

In [ ]:
print(area_data)
print(list(guess_disp_data_all.keys()))
for name, lst in guess_disp_data_all.items():
    array = np.array(lst)
    print(f"{name}: Shape = {array.shape}")


print(list(guess_force_data_all.keys()))
for name, lst in guess_force_data_all.items():
    array = np.array(lst)
    print(f"{name}: Shape = {array.shape}")